# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# План:
1. Открыть файл с данными и изучить его.
2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.
3. Исследовать качество разных моделей, меняя гиперпараметры. Выводы исследования.
4. Проверка качества модели на тестовой выборке.
5. Дополнительно: проверка модели на вменяемость. 

## Откройте и изучите файл

In [3]:
import pandas as pd
import numpy as np 

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier

In [4]:
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


### Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
 - сalls — количество звонков,
 - minutes — суммарная длительность звонков в минутах,
 - messages — количество sms-сообщений,
 - mb_used — израсходованный интернет-трафик в Мб,
 - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
df['is_ultra'].mean()

0.30647168637212197

## Разбейте данные на выборки

In [7]:
features = df.drop('is_ultra',axis = 1) 
target = df['is_ultra']

features_train, features_temp, target_train, target_temp  = train_test_split(features, target, test_size=0.40, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_temp,target_temp, test_size=0.50, random_state=12345)

print('Размер обучающей выборки:',features_train.shape[0])
print('Размер валидационной выборки:',features_valid.shape[0])
print('Размер тестовой выборки:',features_test.shape[0])

Размер обучающей выборки: 1928
Размер валидационной выборки: 643
Размер тестовой выборки: 643


## Исследуйте модели

Целевой признак категориальный, решается задача классификации. Рассмотрим три вида модели:
- дерево решений
- случайный лес
- логистическую регрессию

### Дерево решений

In [8]:
best_res = 0
best_depth = 0

for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if best_res<result:
        best_res = result
        best_depth = depth
print('Лучший результат точности: {}, при max_depth = {}'.format(best_res,best_depth))

Лучший результат точности: 0.7853810264385692, при max_depth = 3


### Cлучайный лес

In [9]:
best_res = 0
best_est = 0
best_depth = 0
for est in range(10,100,10):
    for depth in range(1,11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth).fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if best_res<result:
            best_res = result
            best_est = est
            best_depth = depth
print('Лучший результат точности: {}, при n_estimators = {}, max_depth = {}'.format(best_res,best_est,best_depth))

Лучший результат точности: 0.8087091757387247, при n_estimators = 40, max_depth = 8


### Логистическая регрессия

In [10]:
model = LogisticRegression(random_state=12345,solver='lbfgs', multi_class='auto') 
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
result

0.7107309486780715

Вывод: Идеальная модель это случайный лес с параметрами: n_estimators = 40, max_depth = 8.
Нашел вот такой вариант перебора гиперпараметров, работает долго. Взято из этой статьи, метод "случайного поиска": https://www.machinelearningmastery.ru/hyperparameters-optimization-526348bb8e2d/

In [11]:
random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(10, 1200, 10, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(151, 1200, 10, dtype = int))}

clf = RandomForestClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 80, 
                               cv = 4, verbose= 1, random_state= 12345, n_jobs = -1)
model.fit(features_train,target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('Самая высокая точность, которую удалось достигнуть на валидационной выборке:',result)
model.best_params_

Fitting 4 folds for each of 80 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed: 11.2min finished


Самая высокая точность, которую удалось достигнуть на валидационной выборке: 0.7962674961119751


{'n_estimators': 966,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 10,
 'criterion': 'gini'}

Вывод тот же, что и в практикуме: время и точность обратно пропорциональны. Модель "Случайный лес" - самая точная, а логическая регрессия - самая быстрая.
Перебор гиперпараметров дело очень долгое, рекомендую уменьшить кол-во итераций n_iter, чтоб быстрее было, иначе не дождаться сдачи(20 минут перебора). Не до конца разобрался со всеми параметрами и возможностями. Каждый раз разные результаты, это понятно, но бывают ниже чем при простом переборе по значениям n_estimators и max_depth - это не понятно. При чем делал 4 цикла подряд, с 4мя параметрам: max_depth, n_estimators, min_samples_leaf и min_samples_split было быстрее, точнее, но показалось не красиво.

## Проверьте модель на тестовой выборке

In [12]:
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
result

0.8040435458786936

Хотя на тестовой модели все работает не плохо.

## (бонус) Проверьте модели на адекватность

In [13]:
predictions_test.mean()

0.2317262830482115

Соотношение осталось в принципе тем же, разница в 7%. Модель адекватна.

In [17]:
model = DummyClassifier(strategy='most_frequent', random_state=0)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
result

0.6842923794712286

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
